In [ ]:
import importlib, attack_utils, data_utils, model_utils, model_architectures, general_utils, custom_layer_utils
import lite_model_utils

In [ ]:
importlib.reload(data_utils)
from data_utils import Dataset
import numpy as np

data = Dataset(data_name="Affectnet-Dataset", preferred_size=(128, 128))
data.load_test_data()

In [10]:
importlib.reload(attack_utils)
importlib.reload(model_utils)
importlib.reload(model_architectures)
from model_utils import Keras_Model
from model_architectures import cnn_by_atif
import tensorflow as tf

path="results/"
model_name = 'tf_model_hassan_quantized'
model = tf.keras.models.load_model("results/models/"+model_name+".h5")
keras_model = Keras_Model(path, data=data, model=model, model_architecture=cnn_by_atif)
keras_model.model_name = model_name
keras_model.verbose = False
keras_model.prepare_logits_model()
# keras_model.model.evaluate(data.x_test, data.y_test)
del model

In [16]:
importlib.reload(lite_model_utils)
from lite_model_utils import Lite_Model

lite_model = Lite_Model(keras_model)
lite_model.verbose=False

In [20]:
attack_name = 'pgd'
epsilons = [0.01]

for epsilon in epsilons:
    # print("***************Epsilon: " + str(epsilon) + "*********************")
    # print("Evaluating the vanilla model.")
    # keras_model.perform_adversarial_analysis(attack_name=attack_name,
    #                                          epsilon=epsilon)

    # lite_names = ['dynamic', 'float16', 'integer']
    lite_names = ['float16', 'integer']
    for lite_name in lite_names:
        print("Evaluating tflite " + lite_name + " model.")
        lite_model.prepare_tflite_model(lite_name)
        lite_model.perform_adversarial_analysis(attack_name=attack_name,
                                                 epsilon=epsilon)

Evaluating tflite float16 model.
Float16 Quantization
INFO:tensorflow:Assets written to: C:\Users\hassa\AppData\Local\Temp\tmpog47s6qp\assets


INFO:tensorflow:Assets written to: C:\Users\hassa\AppData\Local\Temp\tmpog47s6qp\assets


Evaluating against the pgd attack.
4/4 [==============================] - 0s 5ms/step - loss: 4.7158 - accuracy: 0.1100
Accuracy before the PGD attack:  0.8799999952316284
Accuracy after the PGD attack: 0.10999999940395355
4/4 [==============================] - 0s 5ms/step - loss: 17.2712 - accuracy: 0.0100
(100, 128, 128, 3)
Evaluating for quantization level:  2
Evaluating for quantization level:  4
Evaluating for quantization level:  8
Evaluating for quantization level:  16
Evaluating for quantization level:  255
Summary
Clean accuraces:  [0.6610169410705566, 0.8414182662963867, 0.9059029817581177, 0.9121371507644653, 0.9053184986114502]
Adversarial accuracies:  [0.49000000953674316, 0.25999999046325684, 0.019999999552965164, 0.009999999776482582, 0.009999999776482582]
Evaluating tflite integer model.
Integer Quantization
INFO:tensorflow:Assets written to: C:\Users\hassa\AppData\Local\Temp\tmpm1r4nxob\assets


INFO:tensorflow:Assets written to: C:\Users\hassa\AppData\Local\Temp\tmpm1r4nxob\assets


Evaluating against the pgd attack.
4/4 [==============================] - 0s 5ms/step - loss: 5.0653 - accuracy: 0.0700
Accuracy before the PGD attack:  0.8899999856948853
Accuracy after the PGD attack: 0.07000000029802322
4/4 [==============================] - 0s 4ms/step - loss: 18.1896 - accuracy: 0.0000e+00
(100, 128, 128, 3)
Evaluating for quantization level:  2
Evaluating for quantization level:  4
Evaluating for quantization level:  8
Evaluating for quantization level:  16
Evaluating for quantization level:  255
Summary
Clean accuraces:  [0.6594583988189697, 0.8418079018592834, 0.9074615240097046, 0.9115527272224426, 0.9047340750694275]
Adversarial accuracies:  [0.5199999809265137, 0.2800000011920929, 0.0, 0.0, 0.0]


In [12]:
from custom_layer_utils import Random_Quantization

batch_size=20
k_model = tf.keras.Sequential()
k_model.add(Random_Quantization(batch_size=batch_size))
for layer in keras_model.model.layers:
  k_model.add(layer)
k_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Epoch 1/6
1550/1550 [==============================] - 16s 11ms/step - loss: 0.8813 - accuracy: 0.6954 - val_loss: 0.7152 - val_accuracy: 0.7520
Epoch 2/6
1550/1550 [==============================] - 14s 9ms/step - loss: 0.7900 - accuracy: 0.7251 - val_loss: 0.6499 - val_accuracy: 0.7750
Epoch 3/6
1550/1550 [==============================] - 15s 10ms/step - loss: 0.7249 - accuracy: 0.7444 - val_loss: 0.6118 - val_accuracy: 0.7956
Epoch 4/6
1550/1550 [==============================] - 15s 10ms/step - loss: 0.6965 - accuracy: 0.7573 - val_loss: 0.6626 - val_accuracy: 0.7810
Epoch 5/6
1550/1550 [==============================] - 15s 10ms/step - loss: 0.6599 - accuracy: 0.7698 - val_loss: 0.5602 - val_accuracy: 0.8032
Epoch 6/6
1550/1550 [==============================] - 15s 9ms/step - loss: 0.6298 - accuracy: 0.7830 - val_loss: 0.5108 - val_accuracy: 0.8276


In [16]:
k_model.fit(data.x_train[:31000], data.y_train[:31000],
            epochs=6, batch_size=batch_size,
            validation_data=(data.x_test[:5000], data.y_test[:5000]))

keras_model.model.evaluate(data.x_test, data.y_test)

Epoch 1/6
1550/1550 [==============================] - 16s 10ms/step - loss: 0.5915 - accuracy: 0.7925 - val_loss: 0.5108 - val_accuracy: 0.8284
Epoch 2/6
1550/1550 [==============================] - 16s 10ms/step - loss: 0.5675 - accuracy: 0.8039 - val_loss: 0.5408 - val_accuracy: 0.8092
Epoch 3/6
1550/1550 [==============================] - 16s 10ms/step - loss: 0.5478 - accuracy: 0.8106 - val_loss: 0.5100 - val_accuracy: 0.8190
Epoch 4/6
1550/1550 [==============================] - 15s 10ms/step - loss: 0.5290 - accuracy: 0.8185 - val_loss: 0.5040 - val_accuracy: 0.8284
Epoch 5/6
1550/1550 [==============================] - 17s 11ms/step - loss: 0.5084 - accuracy: 0.8246 - val_loss: 0.4083 - val_accuracy: 0.8594
Epoch 6/6
1550/1550 [==============================] - 15s 10ms/step - loss: 0.4833 - accuracy: 0.8340 - val_loss: 0.4241 - val_accuracy: 0.8524


In [17]:
keras_model.model.save('results/models/tf_model_hassan_quantized.h5')

161/161 [==============================] - 1s 8ms/step - loss: 0.2757 - accuracy: 0.9053


[0.27573275566101074, 0.9053184986114502]